# TESTING CODE

Model Used: CRF
Features: Previous and next 2 words, 

Evalutation:
1. Precision: 0.872     
2. Recall: 0.875     
3. F1 Score: 0.873 

In [43]:
test_file = codecs.open('./final_test.txt', 'r', encoding="utf-8")
testinput = test_file.read()

In [44]:
X_test,y_test = prepareData(testinput)

In [35]:
# print(len(X_test))
# print(len(y_test))

In [56]:
def test_dev_data(X_test,y_test):
    crf = pickle.load(open('./annCorra_crf_pos_model', 'rb'))
    y_pred=crf.predict(X_test)
    print("F1 score on Test Data ")
    print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
    print("F1 score on Training Data ")
    y_pred_train=crf.predict(X_train)
    print(metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_))

    ### Look at class wise score
    print(metrics.flat_classification_report(
        y_test, y_pred, labels=crf.classes_, digits=3
    ))

In [57]:
test_dev_data(X_test, y_test)

F1 score on Test Data 
0.8726997179430607
F1 score on Training Data 
0.970616867191031
                precision    recall  f1-score   support

          ccof      0.688     0.674     0.681      2490
          rsym      0.999     1.000     0.999      2753
            r6      0.836     0.902     0.867      2287
      lwg__psp      0.998     0.999     0.999      8143
          nmod      0.631     0.553     0.589       698
     nmod__adj      0.930     0.962     0.946      3017
         r6-k2      0.608     0.549     0.577       277
           pof      0.860     0.875     0.867      1508
       pof__cn      1.000     1.000     1.000      2701
            k1      0.768     0.836     0.801      2302
           k7t      0.873     0.901     0.887       835
          main      0.688     0.752     0.719      1804
     lwg__vaux      0.999     0.999     0.999      1647
            k2      0.745     0.693     0.718      1926
            k7      0.824     0.814     0.819      1169
           k7p  

# MODEL CODE

In [47]:
import re
import codecs
import pickle
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn.metrics import accuracy_score

In [24]:
def parse(input_):
    tags = []
    lexicons = []
    lemma = []
    pos= []
    sentences = input_.split("\n\n")
    for sentence in sentences:
        words = sentence.split("\n")
        for word in words:
            tokens = word.split("\t")
            try:
                tags.append(tokens[7])
                lexicons.append(tokens[1])
                pos.append(tokens[4])
                lemma.append(tokens[2])
            except:
                print(sentence)  
        lexicons.append("</s>")
        tags.append("</s>")
        lemma.append("</s>")
        pos.append("</s>")
    
    lexicons.pop()
    tags.pop()
    lemma.pop()
    pos.pop()
    return lexicons, tags, pos, lemma

In [6]:
def features(sentence, index, pos_sentence):
#     print(sentence[index], pos_sentence[index])
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'word':sentence[index],
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'prev_pre_word':'' if index==0 or index==1 else sentence[index-2],
        'next_next_word':'' if index==len(sentence)-1 or index==len(sentence)-2 else sentence[index+2],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'pos': pos_sentence[index]
         }

In [7]:
def prepareData(input):
    lexicons, tags, pos, lemma = parse(input)
#     print(len(lexicons))
    sentences = ' '.join(lexicons).split(' </s> ')
    sentences_pos = ' '.join(pos).split(' </s> ')
    sentences_tags = ' '.join(tags).split(' </s> ')
#     print(sentences_tags)
#     print(len(sentences))
    X=[]
    y=[]
    for sentenceid, sentence in enumerate(sentences):
        words = sentence.split(' ')
        pos = sentences_pos[sentenceid].split(' ')
        X.append([features(words, index, pos) for index in range(len(words))])
    for sentence_tag in sentences_tags:
        words_tag = sentence_tag.split(' ')
        y.append(words_tag)
    return X, y

POS, Chunck, Lemma, Case Marking

In [8]:
def train_CRF(X_train,y_train):   
    crf = CRF(
        algorithm='lbfgs',
        c1=0.01,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    pickle.dump(crf, open("./annCorra_crf_pos_model", 'wb'))

In [40]:
train_file = codecs.open('./final_train.txt', 'r', encoding="utf-8")
traininput = train_file.read()

# dev_file = codecs.open('./final_dev.txt', 'r', encoding="utf-8")
# devinput = dev_file.read()

In [41]:
X_train, y_train = prepareData(traininput)

In [ ]:
# print(len(X_train))
# print((y_train))

In [12]:
train_CRF(X_train, y_train)

In [38]:
# X_dev,y_dev = prepareData(devinput)

In [39]:
# test_dev_data(X_test, y_test)